In [2]:
!ls lightcal

asic_calib_v1.lri     hotpixel.rec          zoom_calib_v0.lri
asic_calib_v1.lri.md5 hotpixel.rec.md5      zoom_calib_v0.lri.md5
calibration.lri       metadata.xml
calibration.lri.md5   uuid.txt


In [3]:
import sys
import os
import struct
from google.protobuf.json_format import MessageToJson
import json
import numpy as np

sys.path.insert(1, 'lri-proto/python_bindings')

# Enumerate your BlockTypes in Python to match the Rust definition
class BlockType:
    LightHeader = 0
    ViewPreferences = 1
    GPSData = 2
    # Add other types as needed

def read_binary_file(filepath):
    with open(filepath, "rb") as file:
        content = file.read()
    return content


def parse_header_mod(data):
    magic = b"LELR"
    if data[:4] != magic:
        raise ValueError("Magic number is incorrect")

    combined_length, message_offset, message_length = struct.unpack('<QQL', data[4:24])
    kind = data[24]

    if kind == BlockType.LightHeader:
        message_type = "lightheader_pb2.LightHeader"
    elif kind == BlockType.ViewPreferences:
        message_type = "view_preferences_pb2.ViewPreferences"
    elif kind == BlockType.GPSData:
        message_type = "gps_data_pb2.GPSData"
    else:
        raise ValueError(f"Unknown block type {kind}")

    return combined_length, message_offset, message_length, message_type
    

from view_preferences_pb2 import ViewPreferences
from gps_data_pb2 import GPSData
from lightheader_pb2 import LightHeader

In [4]:
file_path = "lightcal/calibration.lri"
binary_content = read_binary_file(file_path)
binary_content_temp = binary_content
previous = 0 
block_info_json = []
json_light_header = []
block_info = []

In [5]:
 header_info = (parse_header_mod(binary_content_temp))

In [6]:
header_info

(32864, 32, 32832, 'lightheader_pb2.LightHeader')

In [7]:
if(header_info[3]=="lightheader_pb2.LightHeader"):
        light_header = LightHeader()
        light_header.ParseFromString(binary_content_temp[header_info[1]:header_info[1]+header_info[2]])
        

In [15]:
light_header_json = json.loads(MessageToJson(light_header))
light_header_json

{'deviceUniqueIdLow': '402987039273216048',
 'deviceUniqueIdHigh': '13711472708449499160',
 'deviceModelName': 'L16',
 'moduleCalibration': [{'cameraId': 'A1',
   'geometry': {'mirrorType': 'NONE',
    'perFocusCalibration': [{'focusDistance': 818.0,
      'intrinsics': {'kMat': {'x00': 3374.4456,
        'x01': 0.0,
        'x02': 2111.0083,
        'x10': 0.0,
        'x11': 3374.4456,
        'x12': 1591.6075,
        'x20': 0.0,
        'x21': 0.0,
        'x22': 1.0}},
      'sensorTemp': 25,
      'focusHallCode': 9673.0},
     {'focusDistance': 1500.0,
      'intrinsics': {'kMat': {'x00': 3374.4456,
        'x01': 0.0,
        'x02': 2111.0083,
        'x10': 0.0,
        'x11': 3374.4456,
        'x12': 1591.6075,
        'x20': 0.0,
        'x21': 0.0,
        'x22': 1.0}},
      'sensorTemp': 27,
      'focusHallCode': 10131.0},
     {'focusDistance': 818.0,
      'extrinsics': {'canonical': {'rotation': {'x00': 1.0,
         'x01': 0.0,
         'x02': 0.0,
         'x10': 0

In [17]:
import json
with open('lightcal/data.json', 'w') as f:
    json.dump(light_header_json, f, indent=4)